__Purpose:__ Introduce Federated Learning, specifically by implementing FedAveraging on our dataset and moving on to more advanced methods.  Start by modifying the Simulations code, worry about (a)synchronicity later.
<br>
1. The dec matrix is the weights to pass back an forth (I think), although it comes out of SmoothBatch first
1. We are assuming we can test on the second half (updates 10-19ish) since learning should be complete by then!
1. Scipy.optimize.minimize() runs many iters to fully minimize its cost function.  You can change it to run as many iters as you'd like, although AFAIK you won't know how many it takes to converge.  But this is still a good set up for FL.
1. Hmm minimize() is doing BFGS rn and not SGD... not sure if that matters really.  Could probably implement SGD on my own or find it.  BFGS is 2nd order but we don't have a lot of parameters, I don't think.  Plus we can (already have?) solved analytically for the Hessian.

In [1]:
import pandas as pd
import os
import numpy as np
#from numpy.matlib import repmat
#from matplotlib import pyplot as plt
#from scipy.signal import detrend, firwin, freqz, lfilter
#from sklearn.model_selection import train_test_split, ShuffleSplit
from scipy.optimize import minimize, least_squares
import copy
from itertools import permutations

In [2]:
from experiment_params import *
from simulations import *
import time
# Do the below if you're in the pytch environment
#import pickle5 as pickle
import pickle

# Reminder of Conditions Order

NOTE: 

* **CONDITIONS** = array(['D_1', 'D_2', 'D_5', 'D_6', 'D_3', 'D_4', 'D_7','D_8']
* **LEARNING RATES:** alpha = 0.25 and 0.75; alpha = 0.25 for D1, D2, D5, D6; alpha = 0.75 for D3, D4, D7, D8
* **SMOOTHBATCH:** W_next = alpha*W_old + ((1 - alpha) * W_calc)

* **DECODER INIT:** pos for D1 - D4, neg for D5 - D8

* **PENALTY TERM:** $\lambda_E$ = 1e-6 for all, $\lambda_F$ = 1e-7 for all, $\lambda_D$ = 1e-3 for 1, 3, 5, 7 and 1e-4 for 2, 4, 6, 8 


| DECODER | ALPHA | PENALTY | DEC INIT |
| --- | --- | --- | --- |
| 1 | 0.25 | 1e-3 | + |
| 2 | 0.25 | 1e-4 | + |
| 3 | 0.75 | 1e-3 | + |
| 4 | 0.75 | 1e-4 | + |
| 5 | 0.25 | 1e-3 | - |
| 6 | 0.25 | 1e-4 | - |
| 7 | 0.75 | 1e-3 | - |
| 8 | 0.75 | 1e-4 | - |


## Load Our Data In

In [3]:
t0 = time.time()

with open('Data\continuous_full_data_block1.pickle', 'rb') as handle:
    #refs_block1, poss_block1, dec_vels_block1, int_vel_block1, emgs_block1, Ws_block1, Hs_block1, alphas_block1, pDs_block1, times_block1, conditions_block1 = pickle.load(handle)
    refs_block1, _, _, _, emgs_block1, Ws_block1, _, _, _, _, _ = pickle.load(handle)

#with open('Data\continuous_full_data_block2.pickle', 'rb') as handle:
    #refs_block2, poss_block2, dec_vels_block2, int_vel_block2, emgs_block2, Ws_block2, Hs_block2, alphas_block2, pDs_block2, times_block2, conditions_block2 = pickle.load(handle)
    #refs_block2, _, _, _, emgs_block2, Ws_block2, _, _, _, _, _ = pickle.load(handle)

t1 = time.time()
total = t1-t0  
print(total)

4.751268625259399


In [4]:
# 8 conditions, 20770 data points (only 19 unique sets!), xy, channels
Ws_block1[keys[0]].shape

(8, 20770, 2, 64)

In [5]:
update_ix

array([    0,  1200,  2402,  3604,  4806,  6008,  7210,  8412,  9614,
       10816, 12018, 13220, 14422, 15624, 16826, 18028, 19230, 20432,
       20769])

In [6]:
dec_cond0_user1_update0 = Ws_block1[keys[0]][0,0,:,:]
dec_cond0_user1_update1 = Ws_block1[keys[0]][0,update_ix[1],:,:]
dec_cond0_user1_update2 = Ws_block1[keys[0]][0,update_ix[2],:,:]

print(f"Shape of decoder: {dec_cond0_user1_update0.shape}")
print()
print(f"Total difference between dec0 and dec1: {(dec_cond0_user1_update0 - dec_cond0_user1_update1).sum()}")
print("E.g., as previously shown, the first two decs are the same")
print()
print(f"Total difference between dec0 and dec2: {(dec_cond0_user1_update0 - dec_cond0_user1_update2).sum()}")

Shape of decoder: (2, 64)

Total difference between dec0 and dec1: 0.0
E.g., as previously shown, the first two decs are the same

Total difference between dec0 and dec2: 3.1981579823181594


## Run One Iteration On Above Data and Check Decoders Are the Same
1. Modifying Simulations Code

In [7]:
# Don't run this block, it's just for reference for now
assert(1==0)

# Just 1 person
filtered_signals = emgs_block1[keys[0]][0,:,:]
# Read in the reference positions from the pickle file
cued_target_position = refs_block1[keys[0]][0,:,:]

print(filtered_signals.shape)
print(cued_target_position.shape)

# Previously created random decoder, but we are trying to rerun
#D_0 = np.random.rand(2,64)
D_0 = Ws_block1[keys[0]][0,0,:,:]
D = []
D.append(D_0)

learning_batch = update_ix[1]

# Original simulation inputs
#alpha = .95 # higher alpha means more old decoder (slower update)
#alphaF = 1e-1
#alphaD = 1e-1
# For condition 1:
alpha = .25 # higher alpha means more old decoder (slower update)
# Assuming these are the same as lambda's, the decoder cost penalties
alphaF = 1e-7
alphaD = 1e-3
#where is lambda E?

#def simulation(D,learning_batch,alpha,alphaF=1e-7,alphaD=1e-3,display_info=False,num_iters=False):
display_info=True
num_iters=False

# Should I define this a priori? 
num_updates = 19

# batches the trials into each of the update batch
# Do num_updates-1 because the very last update is only 1 datapoint, the 2nd to last is only 337
for ix in range(num_updates-1):
    #print(ix)
    # For less cluttering when debugging
    #display_info = False
    
    # Instead of using learning_batch, we should get the same results just using update_ix values
    lower_bound = update_ix[ix]
    if ix==(num_updates-1):
        upper_bound = total_datapoints
    else:
        upper_bound = update_ix[ix+1]
        
    s = np.transpose(filtered_signals[lower_bound:upper_bound,:])  # Last working one
    p_intended = np.transpose(cued_target_position[lower_bound:upper_bound,:])  # This is the last working one that runs
    v_intended,p_constrained = output_new_decoder(s,D[-1],p_intended)

    # UPDATE DECODER
    u = copy.deepcopy(s) # u is the person's signal s (64 CHANNELS X TIMEPOINTS)
    q = copy.deepcopy(v_intended) # use cued positions as velocity vectors for updating decoder should be 2 x num_trials
    # emg_windows against intended_targets (trial specific cued target)
    F = copy.deepcopy(u[:,:-1]) # note: truncate F for estimate_decoder
    V = copy.deepcopy(q)

    # initial decoder estimate for gradient descent
    D0 = np.random.rand(2,64)
    # set alphas
    H = np.zeros((2,2))
    # use scipy minimize for gradient descent and provide pre-computed analytical gradient for speed
    if num_iters is False:
        out = minimize(lambda D: cost_l2(F,D,H,V,learning_batch,alphaF,alphaD), D0, method='BFGS', jac=lambda D: gradient_cost_l2(F,D,H,V,learning_batch,alphaF,alphaD), options={'disp': display_info})
    else:
        out = minimize(lambda D: cost_l2(F,D,H,V,learning_batch,alphaF,alphaD), D0, method='BFGS', jac=lambda D: gradient_cost_l2(F,D,H,V,learning_batch,alphaF,alphaD), options={'disp': display_info, 'maxiter':num_iters})

    # reshape to decoder parameters
    W_hat = np.reshape(out.x,(2, 64))

    # DO SMOOTHBATCH
    W_new = alpha*D[-1] + ((1 - alpha) * W_hat)
    D.append(W_new)

AssertionError: 

p_int: (2, 1202) <br>
v_int: (2, 1202) <br>
u: (64, 1202) <br>
q: (2, 1202) <br>
F: (64, 1201) <br>
V: (2, 1202) <br>

# Create Federated Set Up

In [8]:
# given decoder, what is the new position?
def output_new_decoder(s,D,p_intended):
    '''
    s: (64 x (60 timepoints x learning batch size))
    D: (2 x 64) previous D computed or random
    p_intended: (2 x 60 timepoints x learning batch size)
    '''
    # take first trial, random decoder, and do target classification
    v = D@s # actual decoded velocity (2,60)

    # integrate decoded velocities into positions
    p = []
    for ix in range(v.shape[1]):
        p.append(np.sum(v[:,:ix],axis=1))
    p = np.asarray(p).T # actual decoded position (2,60)

    # want error between intended and actual velocity but need to constrain actual velocity to target radius
    p_constrained = np.asarray([constrain_p_actual(p_) for p_ in p.T]).T # constrained, (2,60)
    # compute error between intended and actual position and take derivative to get intended velocity
    v_intended = p_intended - p_constrained # (2,60)
    return v_intended,p_constrained

In [16]:
# python src/emg_fl_main.py --num_users=14, --model=___, --dataset=___, --num_classes=___, --iid=0)

def run_fl_sim(data_path,training_data,labels,epochs=10,num_users=14,C=0.1,local_epochs=10,local_batch_sz=10,lr=0.01,SGD_momentum=0.5,optimizer='sgd',iid=0,unequal=0,stopping_rounds=10,verbose=True,seed=1):
    # Other possible parameters
    #'num_channels'=64,
    #'norm'='batch_norm',    
    # Figure out what dataset to use... all EMG data?
    # Idk how many classes... we are doing regression...
    #parser.add_argument('num_classes', type=int, default=10
    # Explanation kept for these
    # Our application is probably non-IID?
    #parser.add_argument('iid', type=int, default=1,help='Default set to IID. Set to 0 for non-IID.')
    # Our splits are currently equal but irl they would not be
    #parser.add_argument('unequal', type=int, default=0,
    #                    help='whether to use unequal data splits for  \
    #                    non-i.i.d setting (use 0 for equal splits)')
    
    # Probably need to also pass in alphaF/E/D, maybe D_0?
    return


In [17]:
# Different training approaches
def train_1gradstep(w, eta, F, D, H, V, learning_batch, alphaF, alphaD):
        w = w - eta*gradient_cost_l2(F, D, H, V, learning_batch, alphaF, alphaD)
        return w
    
def train_1scipyminstep(w, eta, F, D, H, V, learning_batch, alphaF, alphaD, D0, display_info):
    w = minimize(lambda D: cost_l2(F,D,H,V,learning_batch,alphaF,alphaD), D0, method='BFGS', jac=lambda D: gradient_cost_l2(F,D,H,V,learning_batch,alphaF,alphaD), options={'disp': display_info, 'maxiter':1})
    return w

In [ ]:
class model_super:
    def __init__(self, ID, local_data, w, method, num_steps=1, delay_scaling=5, random_delays=False, download_delay=1, upload_delay=1, local_round=0, lr=0.25, alphaF=1e-7, alphaD=1e-3, verbose=False):
        # Client ID number
        self.ID = ID
        # Linear regression weights AKA the decoder
        self.w = w
        # Which training algorithm to use
        self.method = method
        # Number of gradient steps to take when training (eg amount of local computation)
        self.num_steps = num_steps
        # ML Parameters / Conditions
        self.lr = lr
        self.alphaF = alphaF
        self.alphaD = alphaD
        self.verbose = verbose

In [34]:
temp=-1
while temp==-1:
    # I don't remember if this works for both jupyter notebook and terminal/IDE
    temp_input = input("How many clients are currently running? ")
    try:
        temp = int(temp_input)
        print(temp)
        print(type(temp))
    except:
        print("Please input integers only")

How many clients are currently running? two
Please input integers only
How many clients are currently running? afsdadf
Please input integers only
How many clients are currently running? 343
343
<class 'int'>


In [18]:
# BUILD MODEL
# So for us... just set it to linear regression I guess lol
# I don't know if I need to explicitly define a model object... would make things more legible tho
'''global_model = linear_regression (D@s)'''
class global_model:
    def __init__(self, w, method, C=0.1 global_round=0, ID=0, lr=0.25, alphaF=1e-7, alphaD=1e-3, verbose=False):
        self.w = w
        self.method = method
        self.global_round = global_round
        self.ID = ID
        #w, eta, F, D, H, V, learning_batch, alphaF, alphaD, D0, display_info
        # ML Parameters / Conditions
        self.lr = lr
        self.alphaF = alphaF
        self.alphaD = alphaD
        self.verbose = verbose
        
        # Find a better way to do this, setting to -1 for now so while loop can run
        self.num_avail_clients = -1
        
    def __repr__(self): 
        return f"Global model: {self.ID}\nGlobal Round: {self.global_round}\nTraining Method: {self.method}"
        
    def vanilla_FL_loop(self):
        # These are regular methods that would need to be called on an object...
        
        # Choose fraction C of available clients
        set_num_available_clients()
        choose_clients()
        # Send those clients the current global model
        # Let those clients train
        # Recieve local models from those clients
        # Aggregate learning rates from each local model
        # Aggregate local model weights, weighted by normalized local learning rate
        # Loop is complete, new global decoder is self.w
        
    def get_weights(self):
        return self.w
    
    def set_num_available_clients(self):
        while self.num_avail_clients==-1:
            # I don't remember if this works for both jupyter notebook and terminal/IDE
            temp_input = input("How many clients are currently running? ")
            try:
                self.num_avail_clients = int(temp_input)
            except:
                print("Please input integers only")
    
    def choose_clients(self):
        if self.num_avail_clients > 0:
            num_clients = np.ceil(self.num_avail_clients*self.C)
            print(f"Choosing {num_clients} clients for computation")
            
        else:
            print(f"ERROR: Number of available clients must be greater than 0: {self.num_avail_clients}")
    
    def agg_local_weights(self):
        # Update global round number
        self.global_round += 1
        
        # From McMahan 2017 (vanilla FL)
        # First, sum 
        #self.w = 
        return self.w
    
    
        
        

In [19]:
D_0 = np.random.rand(2,64)
global_model_obj = global_model(D_0, 1)

In [23]:
class client:
    def __init__(self, ID, local_data, w, method, num_steps=1, delay_scaling=5, random_delays=False, download_delay=1, upload_delay=1, local_round=0, lr=0.25, alphaF=1e-7, alphaD=1e-3, verbose=False):
        # Client ID number
        self.ID = ID
        # Local dataset
        self.local_data = local_data
        # Linear regression weights AKA the decoder
        self.w = w
        # Which training algorithm to use
        self.method = method
        # Number of gradient steps to take when training (eg amount of local computation)
        self.num_steps = num_steps
        # Boolean setting whether or not up/download delays should be random or predefined
        self.random_delays = random_delays
        # Scaling from random [0,1] to number of seconds
        self.delay_scaling = delay_scaling
        # Set the delay times
        if self.random_delays: 
            self.download_delay = random()*self.delay_scaling
            self.upload_delay = random()*self.delay_scaling
        else:
            self.download_delay = download_delay
            self.upload_delay = upload_delay
        # Local round number (for asynch FL)
        self.local_round = local_round
        # ML Parameters / Conditions
        self.lr = lr
        self.alphaF = alphaF
        self.alphaD = alphaD
        self.verbose = verbose
        
        #w, eta, F, D, H, V, learning_batch, alphaF, alphaD, D0, display_info
        
        # Hard coded attributes
        # THESE SHOULD NOT CHANGE, SHARED FOR ALL INSTANCES
        num_updates = 19
        starting_update = 10
        self.local_update = starting_update
        # I didn't load this in just copied it in cause it's faster
        update_ix = [0,  1200,  2402,  3604,  4806,  6008,  7210,  8412,  9614, 10816, 12018, 13220, 14422, 15624, 16826, 18028, 19230, 20432, 20769]
        
        # Handle train-test split
        # Should probably create F, D, H, V, learning_batch, alphaF, alphaD
        # Note that these would need to be updated with each update
        
    def __repr__(self): 
        return f"Client (local) model: {self.ID}\nLocal Round: {self.local_round}\nTraining Method: {self.method}"
    
    def training_loop(self):
        #pull_update(new_model) ... this needs to be called on the object?
        set_up_update()
        train_model()
        #send_update() ... this would also need to be called on the object
    
    def simulate_delay(self, incoming):
        '''
        Inputs:
            incoming: [0, 1] --> [upload, download]
            
        Purpose:
            Simulate the random delay associated with ___
        '''
        
        if incoming:
            time.sleep(self.download_delay+random())
        else:
            time.sleep(self.upload_delay+random())
            
    def set_up_update(self):
        lower_bound = update_ix[self.local_round-1]
        upper_bound = update_ix[self.local_round]
        self.learning_batch = upper_bound - lower_bound
        s = np.transpose(self.emg_dataset[lower_bound:upper_bound,:])
        p_intended = np.transpose(self.cued_target_position[lower_bound:upper_bound,:])
        v_intended, p_constrained = output_new_decoder(s, self.w, p_intended)
        
        self.F = s[:,:-1] # note: truncate F for estimate_decoder
        self.V = v_intended
        self.D = self.w
        self.H = np.zeros((2,2))
        # Do I need to return anything since I'm already setting them with self?
    
    def train_model(self):
        for i in range(self.num_steps):
            if self.method==1:
                self.w = train_1gradstep(self.w, self.eta, self.F, self.D, self.H, self.V, self.learning_batch, self.alphaF, self.alphaD)
            elif self.method==2:
                D0 = np.random.rand(2,64)
                self.w = train_1scipyminstep(self.w, self.eta, self.F, self.D, self.H, self.V, self.learning_batch, self.alphaF, self.alphaD, D0, self.verbose)
            else:
                print("Unrecognized method")
            
    def test_model(self):
        # Execute a training loop but don't update D?
        # Then report accuracy in terms of predicted vs ground truth
        pass
    
    def get_update(self, new_model):
        #simulate_delay(incoming=True)
        self.global_model = new_model
        # Update the local round number to reflect the new data
        # I don't think it matters if the update happens on up/download, as long as everyone is consistent
        self.local_round += 1
    
    def set_update(self):
        #simulate_delay(incoming=False)
        return self.local_model

In [31]:
# 'my_data' needs to be changed to a dataframe probably
participant1 = client(1, 'my_data1', D_0, 1)
participant2 = client(2, 'my_data2', D_0, 1)
participant3 = client(3, 'my_data3', D_0, 1)

print(participant1)
# Says no attribute of starting_update... because it's shared I think?
#print(participant1.starting_update)
print(f"P1: {participant1.local_update}")
print()
print("Now let's update the local_update and make sure the change stays local")
participant1.local_update += 1
print(f"P1: {participant1.local_update}")
print(f"P2: {participant2.local_update}")
print(f"P3: {participant3.local_update}")
print("SUCCESS")
print()
print("Now check to make sure that participants created later still have the original update counter")
participant4 = client(4, 'my_data4', D_0, 1)
print(f"P4: {participant4.local_update}")
print("SUCCESS")

Client (local) model: 1
Local Round: 0
Training Method: 1
P1: 10

Now let's update the local_update and make sure the change stays local
P1: 11
P2: 10
P3: 10
SUCCESS

Now check to make sure that participants created later still have the original update counter
P4: 10
SUCCESS


In [ ]:
#import os
#import copy
#import time
#import pickle
#import numpy as np
#from tqdm import tqdm
#from ARJ_update import LocalUpdate, test_inference
#from ARJ_my_models import MLP, CNNMnist, CNNFashion_Mnist, CNNCifar
#from ARJ_utils import get_dataset, average_weights, exp_details

######################################################################

# define paths
#path_project = os.path.abspath('..')
#logger = SummaryWriter('../logs')

# load dataset and user groups
#train_dataset, test_dataset, user_groups = get_dataset(args)
# Change the above to use these:
#'data_path','training_data','labels'

# Set the model to train and send it to device.
# KAI: I think this actually sends it to the GPU, not a "client" in the FL sense
#global_model.to(device)
#global_model.train()
D_0 = np.random.rand(2,64)  #Ws_block1[keys[0]][0,0,:,:]
#global_model_obj = global_model()
print(global_model_obj)

# copy weights
# In this case, our "model" [linear regression] is represented by the matrix D
# And D is also just the weights
global_weights = global_model_obj.get_weights()

# Training
train_loss, train_accuracy = [], []
val_acc_list, net_list = [], []
cv_loss, cv_acc = [], []
print_every = 2
val_loss_pre, counter = 0, 0

# Recall, I am not working with epochs
# I don't think? Maybe replace epochs with iterations
for epoch in tqdm(range(args.epochs)):
    local_weights, local_losses = [], []
    print(f'\n | Global Training Round : {epoch+1} |\n')

    global_model.train()
    m = max(int(args.frac * args.num_users), 1)
    idxs_users = np.random.choice(range(args.num_users), m, replace=False)

    for idx in idxs_users:
        local_model = LocalUpdate(args=args, dataset=train_dataset,
                                  idxs=user_groups[idx], logger=logger)
        w, loss = local_model.update_weights(
            model=copy.deepcopy(global_model), global_round=epoch)
        local_weights.append(copy.deepcopy(w))
        local_losses.append(copy.deepcopy(loss))

    # update global weights
    global_weights = average_weights(local_weights)

    # update global weights
    global_model.load_state_dict(global_weights)

    loss_avg = sum(local_losses) / len(local_losses)
    train_loss.append(loss_avg)

    # Calculate avg training accuracy over all users at every epoch
    list_acc, list_loss = [], []
    global_model.eval()
    for c in range(args.num_users):
        local_model = LocalUpdate(args=args, dataset=train_dataset,
                                  idxs=user_groups[idx], logger=logger)
        acc, loss = local_model.inference(model=global_model)
        list_acc.append(acc)
        list_loss.append(loss)
    train_accuracy.append(sum(list_acc)/len(list_acc))

    # print global training loss after every 'i' rounds
    if (epoch+1) % print_every == 0:
        print(f' \nAvg Training Stats after {epoch+1} global rounds:')
        print(f'Training Loss : {np.mean(np.array(train_loss))}')
        print('Train Accuracy: {:.2f}% \n'.format(100*train_accuracy[-1]))

# Test inference after completion of training
test_acc, test_loss = test_inference(args, global_model, test_dataset)

print(f' \n Results after {args.epochs} global rounds of training:')
print("|---- Avg Train Accuracy: {:.2f}%".format(100*train_accuracy[-1]))
print("|---- Test Accuracy: {:.2f}%".format(100*test_acc))

# Saving the objects train_loss and train_accuracy:
file_name = '../save/objects/{}_{}_{}_C[{}]_iid[{}]_E[{}]_B[{}].pkl'.\
    format(args.dataset, args.model, args.epochs, args.frac, args.iid,
           args.local_ep, args.local_bs)

with open(file_name, 'wb') as f:
    pickle.dump([train_loss, train_accuracy], f)

print('\n Total Run Time: {0:0.4f}'.format(time.time()-start_time))

# PLOTTING (optional)
# import matplotlib
# import matplotlib.pyplot as plt
# matplotlib.use('Agg')

# Plot Loss curve
# plt.figure()
# plt.title('Training Loss vs Communication rounds')
# plt.plot(range(len(train_loss)), train_loss, color='r')
# plt.ylabel('Training loss')
# plt.xlabel('Communication Rounds')
# plt.savefig('../save/fed_{}_{}_{}_C[{}]_iid[{}]_E[{}]_B[{}]_loss.png'.
#             format(args.dataset, args.model, args.epochs, args.frac,
#                    args.iid, args.local_ep, args.local_bs))
#
# # Plot Average Accuracy vs Communication rounds
# plt.figure()
# plt.title('Average Accuracy vs Communication rounds')
# plt.plot(range(len(train_accuracy)), train_accuracy, color='k')
# plt.ylabel('Average Accuracy')
# plt.xlabel('Communication Rounds')
# plt.savefig('../save/fed_{}_{}_{}_C[{}]_iid[{}]_E[{}]_B[{}]_acc.png'.
#             format(args.dataset, args.model, args.epochs, args.frac,
#                    args.iid, args.local_ep, args.local_bs))